In [53]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3

In [54]:
trading_list = {
    "IOC": {"price": 102.80, "leverage": 9, "reward": 1.5, "risk": 0.75},
    "TATAPOWER": {"price": 109.35, "leverage": 8, "reward": 1.5, "risk": 0.75},
    "ONGC": {"price": 115.10, "leverage": 9, "reward": 1.5, "risk": 0.75},
    "GAIL": {"price": 161.05, "leverage": 9, "reward": 1.5, "risk": 0.75},
    "ITC": {"price": 203.25, "leverage": 9, "reward": 2, "risk": 1},
    "HINDPETRO": {"price": 262.25, "leverage": 10, "reward": 1.5, "risk": 0.75},
    "DLF": {"price": 265.30, "leverage": 8, "reward": 1.5, "risk": 0.75},
    "AMBUJACEM": {"price": 309.05, "leverage": 9, "reward": 2, "risk": 1},
    "TATAMOTORS": {"price": 315.55, "leverage": 7, "reward": 2, "risk": 1},
    "HINDZINC": {"price": 317.10, "leverage": 10, "reward": 2, "risk": 1},
    "SBIN": {"price": 364.65, "leverage": 9, "reward": 2, "risk": 1},
    "WIPRO": {"price": 507.60, "leverage": 9, "reward": 2, "risk": 1},
    "DABUR": {"price": 529.20, "leverage": 9, "reward": 2, "risk": 1},
    "BHARTIARTL": {"price": 561.75, "leverage": 9, "reward": 2, "risk": 1},
    "ICICIBANK": {"price": 594.75, "leverage": 9, "reward": 2, "risk": 1},
    "TATACONSUM": {"price": 640.05, "leverage": 9, "reward": 2, "risk": 1},
    "VEDL": {"price": 294.75, "leverage": 9, "reward": 2, "risk": 1},
    "ZEEL": {"price": 191.0, "leverage": 9, "reward": 2, "risk": 1},
    
}

ticker_list = list(trading_list.keys())
data_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\"

In [55]:
class TickerManager:

    def __init__(self,name):
        self.name =name 
        
    def connectDB(self):
        self.db = sqlite3.connect(self.name)

    def tokenLookup(self, symbolList):
        filename = data_directory+"NSE_Instruments.csv"
        df = pd.read_csv(filename)
        tokenList = []
        for symbol in symbolList:
            instrument = int(df[df.tradingsymbol == symbol].instrument_token.values[0])
            tokenList.append({"ticker":symbol,"token":instrument})            
        return tokenList

    def getTicker(self,instrument,time='30S',startTime=None,endTime=None):
        self.connectDB()
        query = '''SELECT * FROM TOKEN_%s ''' % instrument

        if (startTime is not None and endTime is not None):
            query += ''' WHERE `ts` >= '%s' ''' % startTime
            query += ''' and `ts` <= '%s' ''' %  endTime

        query += ''' ;'''
        print(query)
        data = pd.read_sql( query, con=self.db)
        if(len(data) == 0):
            return None
        data = data.set_index(['ts'])
        data.index = pd.to_datetime(data.index)
        ticks = data.loc[:, ['price']]
        df = ticks['price'].resample(time).ohlc().dropna()
        self.close()
        return df
    
    def close(self):
        self.db.close()


In [56]:
def spread(data,ticker):
    data.iplot(kind="spread", keys=["open", "close",],
               title="Open & Close Price Ratio Chart")

In [57]:
def candle_graph(data,ticker):
    qf=cf.QuantFig(data,rangeslider=True,title='EMA ',legend='top',name=ticker)
    qf.add_volume()
    qf.iplot()

In [58]:
class StatisticsManager:

    def __init__(self):
        directory = data_directory + "db\\nikhil\\"
        if not os.path.exists(directory):
            os.makedirs(directory)
        self.name =directory+ "statistics.db"        
        self.createStatistics()

    def connectDB(self):
        self.db = sqlite3.connect(self.name)

    def createStatistics(self):
        self.connectDB()
        cursor = self.db.cursor()
        query = "CREATE TABLE IF NOT EXISTS Statistics (id INTEGER PRIMARY KEY, open_time DATETIME NOT NULL, symbol TEXT NOT NULL, algorithm TEXT NOT NULL, number_of_shares INTEGER, type TEXT NOT NULL, current REAL(15,5), reward REAL(15,5), risk REAL(15,5), close REAL(15,5), status TEXT NOT NULL, close_time DATETIME)"
        cursor.execute(query)
        try:
            self.db.commit()
        except Exception as e:
            print(e)
            self.db.rollback()
        self.close()

    def insertStatistics(self,data):
        self.connectDB()
        cursor = self.db.cursor()
        vals = [data['open_time'],
                data['symbol'],
                data['algorithm'],
                data['count'],
                data['type'],
                data['current'],
                data['reward'],
                data['risk'],
                data['close'],
                data['status'],
                data['close_time']
                ]
        query = "INSERT INTO Statistics(open_time, symbol, algorithm, number_of_shares, type, current, reward, risk, close, status, close_time) VALUES (?,?,?,?,?,?,?,?,?,?,?)"
        cursor.execute(query, vals)
        try:
            self.db.commit()
        except Exception as e:
            print(e)
            self.db.rollback()
        self.close()

    def updateStatistics(self, data):
        self.connectDB()
        cursor = self.db.cursor()
        vals = [data['status'],
                data['close'],
                data['close_time'],
                data['symbol'],
                data['algorithm'],
                data['type']
                ]
        query = "UPDATE Statistics  SET status = ?, close =? , close_time = ? WHERE symbol = ? AND algorithm = ? AND type=? AND status = 'open' "
        cursor.execute(query, vals)
        try:
            self.db.commit()
        except Exception as e:
            print(e)
            self.db.rollback()
        self.close()


    def close(self):
        self.db.close()

    def deleteDB(self,db):
        if os.path.exists(db):
            os.remove(db)
        else:
            print("The file does not exist")

In [59]:
class Candle(enum.Enum):
    red = 1
    green = 2
    doji = 3

In [60]:
class Trend(enum.Enum):
    strong_uptrend = 1
    uptrend = 2
    strong_downtrend = 3
    downtrend = 4
    uptrend_reversal = 5
    downtrend_reversal = 6
    strong_uptrend_reversal = 7
    strong_downtrend_reversal = 8
    sidetrend = 9
       

In [61]:
class MoneyManager:
    def __init__(self):
        self.parallelTrade = 2
        self.wallet = 665
        self.leverage = 6
        self.tradingList = {}


    def getNumberofShare(self,price):
        return self.calculateShare(price,self.leverage)


    def getNumberFromTicker(self,ticker):
        data = self.tradingList.get(ticker, None)
        if data["price"] == None:
            return -1
        return self.calculateShare(data["price"],self.leverage)


    def calculateShare(self,price,leverage):
        return round((self.wallet * leverage) / (price * self.parallelTrade)) - 1

In [62]:
class CandleHelper:

    def __init__(self, count=4):
        self.count = count
        self.index = 0

    def is_doji_candle(self, candle):
        return candle == Candle.doji

    def is_green_candle(self, candle):
        return candle == Candle.green

    def is_red_candle(self, candle):
        return candle == Candle.red

    def get_last_candle(self, data):
        if data['close'].values > data['open'].values:
            return Candle.green
        if data['close'].values < data['open'].values:
            return Candle.red
        return Candle.doji
    
    def get_current_data(self,data):
        candle = self.get_last_candle(data)
        spread = 0
        head = 0
        tail = 0
        if self.is_red_candle(candle):
            spread = data['open'].values[0]  - data['close'].values[0] 
            head = data['high'].values[0]  - data['open'].values[0] 
            tail = data['close'].values[0]  - data['low'].values[0] 
        elif self.is_green_candle(candle):
            spread = data['close'].values[0]  - data['open'].values[0] 
            head = data['high'].values[0]  - data['close'].values[0] 
            tail = data['open'].values[0]  - data['low'].values[0] 
        else:
            spread = data['close'].values[0]  - data['open'].values[0] 
            head = data['high'].values[0]  - data['close'].values[0] 
            tail = data['open'].values[0]  - data['low'].values[0]
        return candle.value,round(spread,2),round(head,2),round(tail,2)

    def get_candle_details(self, data):
        candle_data = data.tail(4)
        current = self.get_last_candle(candle_data.tail(1))
        previous = self.get_last_candle(candle_data.tail(2).head(1))
        last_two = self.get_last_candle(candle_data.tail(3).head(1))
        last_three = self.get_last_candle(candle_data.head(1))
        return current, previous, last_two, last_three

    def trend_strength(self, data, trend):
        candle_data = data.tail(4)
        current = candle_data.tail(1)
        previous = candle_data.tail(2).head(1)
        last_two = candle_data.tail(3).head(1)
        last_three = candle_data.head(1)
        reversal_value = 0
        if trend == Trend.strong_uptrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.downtrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.downtrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_uptrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        else:
            trend_value = 0
            reversal_value = 0
        return trend_value, reversal_value

    def get_trends(self, data):
        current, previous, last_two, last_three = self.get_candle_details(data)
        trend = Trend.sidetrend
        if (self.is_green_candle(current) or self.is_doji_candle(current)) and (
                self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend
                else:
                    trend = Trend.uptrend
        elif (self.is_red_candle(current) or self.is_doji_candle(current)) and (
                self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend
                else:
                    trend = Trend.downtrend
        elif self.is_red_candle(current) and (self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend_reversal
                else:
                    trend = Trend.uptrend_reversal
        elif self.is_green_candle(current) and (self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend_reversal
                else:
                    trend = Trend.downtrend_reversal
        else:
            trend = Trend.sidetrend
        trend_value, reversal_value = self.trend_strength(data, trend)
        return trend, trend_value, reversal_value

    def get_next_candle(self,data):
        candle_data = data.tail(1)
        self.index +=1
        return self.get_last_candle(candle_data)
    
    def candle_analysis(self,data,count=8):        
        candle_data = data.tail(count) 
        current_spread =0
        current_head =0
        current_tail =0
        total_spread =0
        total_head =0
        total_tail =0
        green_candle_count = 0
        green_spread =0
        green_head =0
        green_tail =0
        red_candle_count = 0
        red_spread =0
        red_head =0
        red_tail =0
        for index, row in candle_data.iterrows(): 
            trade_data = pd.DataFrame(columns = ['open', 'high', 'low','close'])
            row_data = {}
            row_data["open"] = row["open"]
            row_data["close"] = row["close"]
            row_data["low"] = row["low"]
            row_data["high"]= row["high"]           
            trade_data = trade_data.append(row_data, ignore_index=True)
            candle,spread,head,tail = self.get_current_data(trade_data)
            
            if self.is_green_candle(candle):
                green_candle_count +=1
                green_spread += spread
                green_head +=head/spread
                green_tail +=tail/spread
                
            elif self.is_red_candle(candle):
                red_candle_count +=1
                red_spread += spread
                red_head +=head/spread
                red_tail +=tail/spread  
                
            current_spread = spread
            current_head =head/spread
            current_tail =tail/spread
            total_spread += spread
            total_head +=head/spread
            total_tail +=tail/spread            
        
        average_spread = total_spread / count
        average_green_spread = 0
        average_red_spread = 0
        if green_candle_count >0 and red_candle_count > 0:            
            average_green_spread = green_spread / green_candle_count
            average_red_spread = red_spread / red_candle_count                
            return round(average_spread,2),round(average_green_spread,2),round(average_red_spread,2)
        elif green_candle_count == 0 and red_candle_count == 0:
            return round(average_spread,2),round(average_green_spread,2),round(average_red_spread,2)
        elif green_candle_count == 0:
            average_red_spread = red_spread / red_candle_count     
            return round(average_spread,2),round(average_green_spread,2),round(average_red_spread,2)
        else:
            average_green_spread = green_spread / green_candle_count
            return round(average_spread,2),round(average_green_spread,2),round(average_red_spread,2)

In [63]:
class BasePricePattern:

    def __init__(self, share, count, reward, risk, algorithm):
        self.date = ""
        self.data = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'traded'])
        self.long_trade_list = []
        self.short_trade_list = []
        self.current_short_price = 0
        self.current_long_price = 0
        self.share = share
        self.count = count
        self.algorithm = algorithm
        self.reward = reward
        self.risk = risk
        self.statistic_manager = StatisticsManager()
        self.candle_helper = CandleHelper()
        self.next_long_pattern = False
        self.next_short_pattern = False
        self.long_open = False
        self.short_open = False

    def createTradeHistoryLong(self):
        data = {'open_time': self.date,
                'symbol': self.share,
                'algorithm': self.algorithm,
                'count': self.count,
                'type': "long",
                'current': self.current_long_price,
                'reward': (self.current_long_price + self.reward),
                'risk': (self.current_long_price - self.risk),
                'close': 0,
                'status': "open",
                'close_time': ""
                }
        
        self.long_trade_list.append(data)
        self.statistic_manager.insertStatistics(data)

    def createTradeHistoryShort(self):
        data = {'open_time': self.date,
                'symbol': self.share,
                'algorithm': self.algorithm,
                'count': self.count,
                'type': "short",
                'current': self.current_short_price,
                'reward': (self.current_short_price - self.reward),
                'risk': (self.current_short_price + self.risk),
                'close': 0,
                'status': "open",
                'close_time': ""
                }
        
        self.short_trade_list.append(data)
        self.statistic_manager.insertStatistics(data)    



    def update_trade_statistic(self, data):
        data = {
            'status': data['status'],
            'close': data['close'],
            'close_time': data['close_time'],
            'symbol': data['symbol'],
            'algorithm': data['algorithm'],
            'type': data['type']
        }
        self.statistic_manager.updateStatistics(data)

    def trade(self, data, date):
        self.date = date
        self.data = self.data.append(data, ignore_index=True)
        
        if self.long_open:
            self.close_long_trade() 
        elif self.next_long_pattern:
            self.long_pattern()
        else:
            self.long_trade()
            
        if self.short_open:
            self.close_short_trade()
        elif self.next_short_pattern:
            self.short_pattern()
        else:
            self.short_trade()

    def long_pattern(self):
        if self.candle_helper.index > 3:
            self.candle_helper.index = 0
            self.next_long_pattern = False
            self.long_trade()
        else:
            candle_data = self.data.tail(1)
            pattern = self.candle_helper.get_next_candle(candle_data)
            if pattern == Candle.green and self.long_trend_analysis() :
                self.candle_helper.index = 0
                self.next_long_pattern = False
                self.long_trade(long_pattern=True)

    def short_pattern(self):
        if self.candle_helper.index > 3:
            self.candle_helper.index = 0
            self.next_short_pattern = False
            self.short_trade()
        else:
            pattern = self.candle_helper.get_next_candle(self.data.tail(1))
            if pattern == Candle.red and self.short_trend_analysis():
                self.candle_helper.index = 0
                self.next_short_pattern = False
                self.short_trade(short_pattern=True)
                
    def long_trend_analysis(self,count=8):               
        if len(self.data) < 9:            
            return False                 
        mom8 = talib.MOM(self.data.tail(9)["close"].values, timeperiod=8)
        mom4 = talib.MOM(self.data.tail(5)["close"].values, timeperiod=4)
        return (mom8[8] > mom4[4] )    

    
    def short_trend_analysis(self,count=8):
        if len(self.data) < 9:            
            return False
        mom8 = talib.MOM(self.data.tail(9)["close"].values, timeperiod=8)
        mom4 = talib.MOM(self.data.tail(5)["close"].values, timeperiod=4)        
        return  (mom8[8] < mom4[4] )
        
    def close_all_trade(self):
        candle_data = self.data.tail(1)
        closing_price = candle_data['close'].values[0]
        if self.long_open:
            data_long = self.long_trade_list[len(self.long_trade_list) - 1]
            if data_long['status'] == "open":
                data_long['close'] = closing_price
                data_long['close_time'] = self.date
                if closing_price > self.current_long_price:
                    data_long['status'] = "profit"
                else:
                    data_long['status'] = "loss"
                self.long_open = False
                self.update_trade_statistic(data_long)
        if self.short_open:
            data_short = self.short_trade_list[len(self.short_trade_list) - 1]
            if data_short['status'] == "open":
                data_short['close'] = closing_price
                data_short['close_time'] = self.date
                if closing_price < self.current_short_price:
                    data_short['status'] = "profit"
                else:
                    data_short['status'] = "loss"
                self.short_open = False
                self.update_trade_statistic(data_short)
            

    def long_trade(self, long_pattern=False):
        pass

    def short_trade(self, short_pattern=False):
        pass


    def close_long_trade(self):
        pass

    def close_short_trade(self):
        pass

In [64]:
class Nikhil(BasePricePattern):

    def __init__(self, share, count, reward, risk,spread_limit =  0.15):
        super().__init__(
            share,
            count,
            reward,
            risk,
            self.__class__.__name__
        )
        self.spread_limit =  spread_limit


    def long_trade(self, long_pattern=False):
        if long_pattern:
            self.current_long_price = self.data["close"].values[0]
            self.data['Nikhil'] = 100
            self.long_open = True
            self.createTradeHistoryLong()
        else:
            trend, trend_value, reversal_value = self.candle_helper.get_trends(self.data.tail(4))
            if trend == Trend.downtrend_reversal or trend == Trend.strong_downtrend_reversal and self.long_trend_analysis():
                self.current_long_price = self.data["close"].values[0]
                self.data['Nikhil'] = 100
                self.long_open = True
                self.createTradeHistoryLong()
            else:
                self.candle_helper.index = 0
                self.next_long_pattern = True
                        
    def short_trade(self, short_pattern=False):
        if short_pattern:
            self.current_short_price = self.data["close"].values[0]
            self.data['Nikhil'] = -100
            self.short_open = True
            self.createTradeHistoryShort()
        else:
            trend, trend_value, reversal_value = self.candle_helper.get_trends(self.data.tail(4))
            if trend == Trend.uptrend_reversal or trend == Trend.strong_downtrend_reversal and self.short_trend_analysis():
                self.current_short_price = self.data["close"].values[0]
                self.data['Nikhil'] = -100
                self.short_open = True
                self.createTradeHistoryShort()
            else:
                self.candle_helper.index = 0
                self.next_short_pattern = True

    def close_long_trade(self):
        candle_data = self.data.tail(1)
        closing_price = candle_data['close'].values[0]
        spread = round(abs(candle_data['open'].values[0]  - closing_price),2)
        pattern = self.candle_helper.get_next_candle(candle_data)
        average_spread,average_green_spread,average_red_spread = self.candle_helper.candle_analysis(self.data)
        if (pattern == Candle.red and (spread >= average_spread or spread >= average_green_spread)) or closing_price <= self.current_long_price - self.risk :
            data_long = self.long_trade_list[len(self.long_trade_list) - 1]
            if data_long['status'] == "open":
                data_long['close'] = closing_price
                data_long['close_time'] = self.date
                if closing_price > self.current_long_price:
                    data_long['status'] = "profit"
                else:
                    data_long['status'] = "loss"
                self.long_open = False
                self.update_trade_statistic(data_long)
                
    def close_short_trade(self):
        candle_data = self.data.tail(1)
        closing_price = candle_data['close'].values[0]
        spread = round(abs(candle_data['open'].values[0]  - closing_price),2)
        pattern = self.candle_helper.get_next_candle(candle_data)
        average_spread,average_green_spread,average_red_spread = self.candle_helper.candle_analysis(self.data)
        if (pattern == Candle.green and (spread >= average_spread or spread >= average_red_spread)) or closing_price <= self.current_short_price + self.risk :
            data_short = self.short_trade_list[len(self.short_trade_list) - 1]
            if data_short['status'] == "open":
                data_short['close'] = closing_price
                data_short['close_time'] = self.date
                if closing_price < self.current_short_price:
                    data_short['status'] = "profit"
                else:
                    data_short['status'] = "loss"
                self.short_open = False
                self.update_trade_statistic(data_short)

In [65]:
def trade_last_60(ticker, startday, data):
    manager = MoneyManager()
    manager.wallet = 665
    manager.leverage = 6
    manager.parallelTrade = 2
    manager.tradingList = trading_list
    trading = Nikhil(ticker, manager.getNumberFromTicker(ticker),1,0.5)    
    for index, row in data.iterrows():
        row_data = {}
        row_data["open"] = row["open"]
        row_data["close"] = row["close"]
        row_data["low"] = row["low"]
        row_data["high"]= row["high"]
        row_data["volume"] = row["volume"]
        row_data["traded"] = False        
        if len(row_data) > 0:
            trading.trade(row_data,startday)
        else:
            print("hammer trade_data is empty")
    trading.close_all_trade()
    return trading.data

In [66]:
def process_last_60(date,offset):   
    print("day : ",offset)
    startday = date.strftime('%Y-%m-%d-%H:%M:%S')
    directory = data_directory+"last_60\\17_05_2021\\days\\"
    file_name = directory + str(offset) + ".csv"
    data = pd.read_csv(file_name)
    process_data = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'traded'])
    for ticker in ticker_list:        
        daydata = trade_last_60(ticker, startday, data[data["ticker"]==ticker])
        daydata['ticker'] = ticker
        daydata['date'] = startday
        process_data = process_data.append(daydata, ignore_index=True)
    return process_data

In [67]:
def start_last_60_days():
    
    
    start_data = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'traded'])
    for offset in range(1,39):
        offset_data=process_last_60(startday,offset)
        start_data = start_data.append(offset_data, ignore_index=True)        
        startday = startday + timedelta(days=1)
    return start_data

In [68]:
def save(_data):
    directory = data_directory+"test_data\\nikhil\\"
    if not os.path.exists(directory):
        os.makedirs(directory)
    for ticker in ticker_list: 
        file_name = directory+ticker+".csv"
        _data[_data["ticker"]==ticker].to_csv(file_name)

In [69]:
#_data =start_last_60_days()
#save(_data)


In [70]:
db_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\db\\"
ticks_backup3 = "ticks_backup3"
ticks_backup2 = "ticks_backup2"
ticks_backup1 = "ticks_backup1"
ticks_19_05_2021 = "ticks_19_05_2021"
ticks_18_05_2021 = "ticks_18_05_2021"
ticks_17_05_2021 = "ticks_17_05_2021"
ticks_14_05_2021 = "ticks_14_05_2021"

db_list = [ticks_backup3,ticks_backup2,ticks_backup1,ticks_19_05_2021,ticks_18_05_2021,ticks_17_05_2021,ticks_14_05_2021]

In [71]:
manager = TickerManager(db_directory+ticks_backup3+".db")
tocken_list = manager.tokenLookup(ticker_list)

In [80]:
save_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\test_data\\db\\"
manager = TickerManager(db_directory+ticks_backup3+".db")

db_list = [ticks_backup3,ticks_backup2,ticks_backup1]
tocken_list=[{'ticker':'SBIN','token':779521},{'ticker':'VEDL','token':784129},
             {'ticker':'TATAMOTORS','token':884737},{'ticker':'ZEEL','token':975873}]

directory_30 = save_directory + "db_30\\"
directory_45 = save_directory + "db_45\\"
directory_60 = save_directory + "db_60\\"
if not os.path.exists(directory_30):
    os.makedirs(directory_30)
if not os.path.exists(directory_45):
    os.makedirs(directory_45)
if not os.path.exists(directory_60):
    os.makedirs(directory_60)
for name in db_list:
    manager.name = db_directory+name+".db"
    db_data_30 = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])
    db_data_45 = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])
    db_data_60 = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])
    for token in tocken_list:
        data_30 = manager.getTicker(token["token"],time='30S')
        data_30['ticker'] = token["ticker"]
        db_data_30 = db_data_30.append(data_30, ignore_index=True)
        
        data_45 = manager.getTicker(token["token"],time='45S')
        data_45['ticker'] = token["ticker"]
        db_data_45 = db_data_45.append(data_45, ignore_index=True)
        
        data_60 = manager.getTicker(token["token"],time='60S')
        data_60['ticker'] = token["ticker"]
        db_data_60 = db_data_60.append(data_60, ignore_index=True)
        
    file_name_30 = directory_30+name+".csv"
    db_data_30.to_csv(file_name_30)
    
    file_name_45 = directory_45+name+".csv"
    db_data_45.to_csv(file_name_45)
    
    file_name_60 = directory_60+name+".csv"
    db_data_60.to_csv(file_name_60)
    


SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_975873  ;
SELECT * FROM TOKEN_975873  ;
SELECT * FROM TOKEN_975873  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_975873  ;
SELECT * FROM TOKEN_975873  ;
SELECT * FROM TOKEN_975873  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_779521  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_784129  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_884737  ;
SELECT * FROM TOKEN_884737  ;
SELECT * F

In [81]:
tocken_list=[{'ticker':'SBIN','token':779521},{'ticker':'VEDL','token':784129},
             {'ticker':'TATAMOTORS','token':884737},{'ticker':'ZEEL','token':975873}]
db_list = [ticks_backup3,ticks_backup2,ticks_backup1]
days_to_subtract = len(db_list)
startday = datetime.today() - timedelta(days=days_to_subtract)

for name in db_list:
    print("name : ",name)
    filename = directory_60+name+".csv"
    data = pd.read_csv(filename)
    for token in tocken_list:
        trade_last_60(token["ticker"], startday, data[data["ticker"]==token["ticker"]])
    startday = startday + timedelta(days=1)

name :  ticks_backup3
name :  ticks_backup2
name :  ticks_backup1


In [74]:
file_name_30 = directory_30+"ticks_backup3"+".csv"
data = pd.read_csv(file_name_30)
data

,Unnamed: 0,open,high,low,close,volume,ticker
0,0,365.70,365.80,365.65,365.65,NaN,SBIN
1,1,365.80,365.80,365.40,365.60,NaN,SBIN
2,2,365.50,365.50,365.00,365.05,NaN,SBIN
3,3,365.10,365.20,365.00,365.00,NaN,SBIN
4,4,365.10,365.15,365.00,365.15,NaN,SBIN
...,...,...,...,...,...,...,...
2931,2931,192.55,192.75,192.55,192.75,NaN,ZEEL
2932,2932,192.75,192.75,192.75,192.75,NaN,ZEEL
2933,2933,192.75,192.75,192.75,192.75,NaN,ZEEL
2934,2934,193.00,193.00,193.00,193.00,NaN,ZEEL
